In [1]:
import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
class Classifier(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 8, 3, stride=2),
            nn.ReLU(),
            nn.Conv2d(8, 8*8, 3, stride=2),
            nn.ReLU(),
            nn.AvgPool2d(3, 2),
            nn.Flatten(),
            nn.Dropout(),
            nn.Linear(256, 128), 
            nn.Dropout(),
            nn.Linear(128, 10) 
            # nn.Softmax()
        )

    def forward(self, x):
        y = self.model(x)
        return y
    

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to("cuda")
        y = y.to("cuda")
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [3]:
model = Classifier()

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
loss_fn = nn.CrossEntropyLoss()

In [4]:
model = model.to("cuda")
for i in range(100):
    train(train_dataloader, model, loss_fn, optimizer)
model = model.to("cpu")

loss: 2.324635  [   64/60000]
loss: 2.315197  [ 6464/60000]
loss: 2.293364  [12864/60000]
loss: 2.308164  [19264/60000]
loss: 2.313383  [25664/60000]
loss: 2.298207  [32064/60000]
loss: 2.291450  [38464/60000]
loss: 2.313577  [44864/60000]
loss: 2.308894  [51264/60000]
loss: 2.299623  [57664/60000]
loss: 2.301777  [   64/60000]
loss: 2.301740  [ 6464/60000]
loss: 2.305539  [12864/60000]
loss: 2.295763  [19264/60000]
loss: 2.302462  [25664/60000]
loss: 2.281150  [32064/60000]
loss: 2.294436  [38464/60000]
loss: 2.271961  [44864/60000]
loss: 2.289284  [51264/60000]
loss: 2.294258  [57664/60000]
loss: 2.288925  [   64/60000]
loss: 2.296598  [ 6464/60000]
loss: 2.294809  [12864/60000]
loss: 2.284444  [19264/60000]
loss: 2.305398  [25664/60000]
loss: 2.268583  [32064/60000]
loss: 2.306093  [38464/60000]
loss: 2.253063  [44864/60000]
loss: 2.284512  [51264/60000]
loss: 2.260110  [57664/60000]
loss: 2.282986  [   64/60000]
loss: 2.275275  [ 6464/60000]
loss: 2.273907  [12864/60000]
loss: 2.28

In [5]:
%%timeit
test(test_dataloader, model, loss_fn)

Test Error: 
 Accuracy: 74.9%, Avg loss: 0.695426 

Test Error: 
 Accuracy: 74.9%, Avg loss: 0.695426 

Test Error: 
 Accuracy: 74.9%, Avg loss: 0.695426 

Test Error: 
 Accuracy: 74.9%, Avg loss: 0.695426 

Test Error: 
 Accuracy: 74.9%, Avg loss: 0.695426 

Test Error: 
 Accuracy: 74.9%, Avg loss: 0.695426 

Test Error: 
 Accuracy: 74.9%, Avg loss: 0.695426 

Test Error: 
 Accuracy: 74.9%, Avg loss: 0.695426 

945 ms ± 15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
model = model.to("cpu")

model_int8 = torch.ao.quantization.quantize_dynamic(
    model,  # the original model
    {torch.nn.Linear, torch.nn.Conv2d, torch.nn.AvgPool2d, torch.nn.ReLU},  # a set of layers to dynamically quantize
    dtype=torch.qint8)

In [14]:
%%timeit
test(test_dataloader, model_int8, loss_fn)

Test Error: 
 Accuracy: 75.0%, Avg loss: 0.695496 

Test Error: 
 Accuracy: 75.0%, Avg loss: 0.695496 

Test Error: 
 Accuracy: 75.0%, Avg loss: 0.695496 

Test Error: 
 Accuracy: 75.0%, Avg loss: 0.695496 

Test Error: 
 Accuracy: 75.0%, Avg loss: 0.695496 

Test Error: 
 Accuracy: 75.0%, Avg loss: 0.695496 

Test Error: 
 Accuracy: 75.0%, Avg loss: 0.695496 

Test Error: 
 Accuracy: 75.0%, Avg loss: 0.695496 

339 ms ± 5.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
torch.save(model_int8, "model_post_dynamic_quantized.pt")